In [12]:
!pip install praw
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install textblob
!pip install matplotlib seaborn


In [7]:
import praw
import pandas as pd

# Reddit API credentials
reddit = praw.Reddit(
    client_id='Bg1PYjd4bCnygBrVzK-LsQ',
    client_secret='u3PPcsF7XhhsepKP5Uh_m6LpwneF6Q',
    user_agent="Prem by /u/Over-Fact1365"

)

# Define subreddit and fetch posts
subreddit = reddit.subreddit('stocks')  # Replace 'stocks' with your desired subreddit
posts = subreddit.hot(limit=100)  # Fetch the top 100 "hot" posts

# Process the posts
data = []
for post in posts:
    data.append({
        'Title': post.title,
        'Score': post.score,
        'Comments': post.num_comments,
        'URL': post.url
    })

# Save the data to a CSV
df = pd.DataFrame(data)
df.to_csv('reddit_data.csv', index=False)
print("Data saved to reddit_data.csv")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Data saved to reddit_data.csv


In [9]:
from google.colab import files
files.download('reddit_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import pandas as pd

df = pd.read_csv('reddit_data.csv')

print(df.head())
print(df.isnull().sum())
print(df.describe())


                                               Title  Score  Comments  \
0  r/Stocks Daily Discussion & Fundamentals Frida...     21       424   
1  r/Stocks Daily Discussion Wednesday - Nov 20, ...      7       273   
2  Nvidia Earnings: Beat Across the Board, Strong...    363       193   
3  Target shares plunge 20% after discounter cuts...    536       291   
4                              Snowflake Q3 Earnings     43        11   

                                                 URL  
0  https://www.reddit.com/r/stocks/comments/1grtf...  
1  https://www.reddit.com/r/stocks/comments/1gvmi...  
2  https://www.reddit.com/r/stocks/comments/1gvzy...  
3  https://www.reddit.com/r/stocks/comments/1gvrc...  
4  https://www.reddit.com/r/stocks/comments/1gvzu...  
Title       0
Score       0
Comments    0
URL         0
dtype: int64
            Score    Comments
count  100.000000  100.000000
mean   109.130000   74.950000
std    157.070187   95.321505
min      0.000000    0.000000
25%      9.0

In [24]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import download
import pandas as pd

# Download necessary NLTK resources
download('stopwords')
download('punkt')
download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^A-Za-z\s]", "", text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize the text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words("english")]  # Remove stopwords and lemmatize
    return " ".join(tokens)

# Assuming 'df' is your DataFrame loaded from 'reddit_data.csv'
df = pd.read_csv('reddit_data.csv') # Ensure df is loaded or created

# Apply preprocessing to Title and Content or selftext if available
df['cleaned_title'] = df['Title'].apply(preprocess_text)
# Choose one of the options below based on your data:
# Option 1: If you want to clean and use the 'selftext' column (if it exists)
if 'selftext' in df.columns:
    df['cleaned_content'] = df['selftext'].apply(preprocess_text)
# Option 2: If you want to reuse the cleaned titles for 'cleaned_content'
else:
    df['cleaned_content'] = df['Title'].apply(preprocess_text)

# Check the cleaned data
print(df[['cleaned_title', 'cleaned_content']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                       cleaned_title  \
0    rstocks daily discussion fundamental friday nov   
1             rstocks daily discussion wednesday nov   
2  nvidia earnings beat across board strong guidance   
3  target share plunge discounter cut forecast po...   
4                               snowflake q earnings   

                                     cleaned_content  
0    rstocks daily discussion fundamental friday nov  
1             rstocks daily discussion wednesday nov  
2  nvidia earnings beat across board strong guidance  
3  target share plunge discounter cut forecast po...  
4                               snowflake q earnings  


In [25]:
from textblob import TextBlob

# Sentiment analysis function
def get_sentiment(text):
    if not isinstance(text, str):
        return 0
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # Polarity ranges from -1 (negative) to 1 (positive)

# Calculate sentiment scores
df['title_sentiment'] = df['cleaned_title'].apply(get_sentiment)
df['content_sentiment'] = df['cleaned_content'].apply(get_sentiment)

# Inspect sentiment scores
print(df[['cleaned_title', 'title_sentiment', 'cleaned_content', 'content_sentiment']].head())


                                       cleaned_title  title_sentiment  \
0    rstocks daily discussion fundamental friday nov         0.000000   
1             rstocks daily discussion wednesday nov         0.000000   
2  nvidia earnings beat across board strong guidance         0.433333   
3  target share plunge discounter cut forecast po...         0.000000   
4                               snowflake q earnings         0.000000   

                                     cleaned_content  content_sentiment  
0    rstocks daily discussion fundamental friday nov           0.000000  
1             rstocks daily discussion wednesday nov           0.000000  
2  nvidia earnings beat across board strong guidance           0.433333  
3  target share plunge discounter cut forecast po...           0.000000  
4                               snowflake q earnings           0.000000  


In [26]:
# Combine title and content sentiment
df['average_sentiment'] = (df['title_sentiment'] + df['content_sentiment']) / 2

# Create binary sentiment label
df['sentiment_label'] = df['average_sentiment'].apply(lambda x: 1 if x > 0 else 0)

# Check the final features
print(df[['Score', 'Comments', 'average_sentiment', 'sentiment_label']].head())


   Score  Comments  average_sentiment  sentiment_label
0     21       424           0.000000                0
1      7       273           0.000000                0
2    363       193           0.433333                1
3    536       291           0.000000                0
4     43        11           0.000000                0


In [27]:
!pip install yfinance
import yfinance as yf

# Fetch historical stock data (example: Tesla)
stock_data = yf.download("TSLA", start="2023-01-01", end="2023-12-31")
print(stock_data.head())

# Combine Reddit data and stock data based on timestamps
# Ensure that your Reddit data has timestamps and align them properly.
# You may need to preprocess the 'created_utc' column from Reddit scraping (not included in earlier code).


[*********************100%***********************]  1 of 1 completed

Price                       Adj Close       Close        High         Low  \
Ticker                           TSLA        TSLA        TSLA        TSLA   
Date                                                                        
2023-01-03 00:00:00+00:00  108.099998  108.099998  118.800003  104.639999   
2023-01-04 00:00:00+00:00  113.639999  113.639999  114.589996  107.519997   
2023-01-05 00:00:00+00:00  110.339996  110.339996  111.750000  107.160004   
2023-01-06 00:00:00+00:00  113.059998  113.059998  114.389999  101.809998   
2023-01-09 00:00:00+00:00  119.769997  119.769997  123.519997  117.110001   

Price                            Open     Volume  
Ticker                           TSLA       TSLA  
Date                                              
2023-01-03 00:00:00+00:00  118.470001  231402800  
2023-01-04 00:00:00+00:00  109.110001  180389000  
2023-01-05 00:00:00+00:00  110.510002  157986300  
2023-01-06 00:00:00+00:00  103.000000  220911100  
2023-01-09 00:00:00+00:00 

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Features and target
features = df[['Score', 'Comments', 'average_sentiment']]
target = df['sentiment_label']  # Replace this with actual stock movement labels if available

# Split data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00         2

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [29]:
import joblib

# Save the model
joblib.dump(model, 'stock_prediction_model.pkl')
print("Model saved as stock_prediction_model.pkl")


Model saved as stock_prediction_model.pkl
